Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

In [ ]:
import numpy as np
from monai.apps import MedNISTDataset

# MedNIST Classification Bundle

In this tutorial we'll revisit the bundle replicating [MONAI 101 notebook](https://github.com/Project-MONAI/tutorials/blob/main/2d_classification/monai_101.ipynb) and add more features representing best practice concepts. This will include evaluation and checkpoint saving techniques.

We'll first create a bundle very much like in the previous tutorial with the same metadata and common script file:

In [1]:
%%bash

python -m monai.bundle init_bundle MedNISTClassifier_v2
which tree && tree MedNISTClassifier_v2

MedNISTClassifier_v2
├── configs
│   ├── inference.json
│   └── metadata.json
├── docs
│   └── README.md
├── LICENSE
└── models

3 directories, 4 files


In [2]:
%%writefile MedNISTClassifier_v2/configs/metadata.json

{
    "version": "0.0.1",
    "changelog": {
        "0.0.1": "Initial version"
    },
    "monai_version": "1.2.0",
    "pytorch_version": "2.0.0",
    "numpy_version": "1.23.5",
    "optional_packages_version": {},
    "name": "MedNISTClassifier",
    "task": "MedNIST Classification Network",
    "description": "This is a demo network for classifying MedNIST images by type/modality",
    "authors": "Your Name Here",
    "copyright": "Copyright (c) Your Name Here",
    "data_source": "MedNIST dataset kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic)",
    "data_type": "jpeg",
    "intended_use": "This is suitable for demonstration only",
    "network_data_format": {
        "inputs": {
            "image": {
                "type": "image",
                "format": "magnitude",
                "modality": "any",
                "num_channels": 1,
                "spatial_shape": [64, 64],
                "dtype": "float32",
                "value_range": [0, 1],
                "is_patch_data": false,
                "channel_def": {
                    "0": "image"
                }
            }
        },
        "outputs": {
            "pred": {
                "type": "probabilities",
                "format": "classes",
                "num_channels": 6,
                "spatial_shape": [6],
                "dtype": "float32",
                "value_range": [0, 1],
                "is_patch_data": false,
                "channel_def": {
                    "0": "AbdomenCT",
                    "1": "BreastMRI",
                    "2": "CXR",
                    "3": "ChestCT",
                    "4": "Hand",
                    "5": "HeadCT"
                }
            }
        }
    }
}

Overwriting MedNISTClassifier_v2/configs/metadata.json


As you've likely seen in outputs, there should be a `logging.conf` file in the `configs` directory to set up the Python logger appropriately. This will improve the output we get in the notebook:

In [4]:
%%writefile MedNISTClassifier_v2/configs/logging.conf

[loggers]
keys=root

[handlers]
keys=consoleHandler

[formatters]
keys=fullFormatter

[logger_root]
level=INFO
handlers=consoleHandler

[handler_consoleHandler]
class=StreamHandler
level=INFO
formatter=fullFormatter
args=(sys.stdout,)

[formatter_fullFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s


Writing MedNISTClassifier_v2/configs/logging.conf


We'll change the common file slightly by adding some extra symbols, specifically `bundle_root` which should always be present in bundles. We'll keep `root_dir` since it's used to determine where MedNIST is downloaded to.

In [9]:
%%writefile MedNISTClassifier_v2/configs/common.yaml

# added a few more imports
imports: 
- $import torch
- $import datetime
- $import os

root_dir: .

# use constants from MONAI instead of hard-coding names
image: $monai.utils.CommonKeys.IMAGE
label: $monai.utils.CommonKeys.LABEL
pred: $monai.utils.CommonKeys.PRED

# these are added definitions
bundle_root: .
ckpt_path: $@bundle_root + '/models/model.pt'

# define a device for the network
device: '$torch.device(''cuda:0'')'

# store the class names for inference later
class_names: [AbdomenCT, BreastMRI, CXR, ChestCT, Hand, HeadCT]

# define the network separately, don't need to refer to MONAI types by name or import MONAI
network_def:
  _target_: densenet121
  spatial_dims: 2
  in_channels: 1
  out_channels: 6

# define the network to be the given definition moved to the device
net: '$@network_def.to(@device)'

# define a transform sequence as a list of transform objects instead of using Compose here
train_transforms:
- _target_: LoadImaged
  keys: '@image'
  image_only: true
- _target_: EnsureChannelFirstd
  keys: '@image'
- _target_: ScaleIntensityd
  keys: '@image'
    

Overwriting MedNISTClassifier_v2/configs/common.yaml



## Training

For training we have the same elements again but we'll add a `SupervisedEvaluator` object to track model progress with handlers to save checkpoints. 

In [8]:
%%writefile MedNISTClassifier_v2/configs/train.yaml

max_epochs: 25
learning_rate: 0.00001  # learning rate, again artificially slow
val_interval: 1  # run validation every n'th epoch
save_interval: 1 # save the model weights every n'th epoch

# choose a unique output subdirectory every time training is started, 
output_dir: '$datetime.datetime.now().strftime(@root_dir+''/output/output_%y%m%d_%H%M%S'')'

train_dataset:
  _target_: MedNISTDataset
  root_dir: '@root_dir'
  transform: 
    _target_: Compose
    transforms: '@train_transforms'
  section: training
  download: true

train_dl:
  _target_: DataLoader
  dataset: '@train_dataset'
  batch_size: 512
  shuffle: true
  num_workers: 4

# separate dataset taking from the "validation" section
eval_dataset:
  _target_: MedNISTDataset
  root_dir: '@root_dir'
  transform: 
    _target_: Compose
    transforms: '$@train_transforms'
  section: validation
  download: true

# separate dataloader for evaluation
eval_dl:
  _target_: DataLoader
  dataset: '@eval_dataset'
  batch_size: 512
  shuffle: false
  num_workers: 4

# transforms applied to network output, in this case applying activation, argmax, and one-hot-encoding
post_transform:
  _target_: Compose
  transforms:
  - _target_: Activationsd
    keys: '@pred'
    softmax: true  # apply softmax to the prediction to emphasize the most likely value
  - _target_: AsDiscreted
    keys: ['@label','@pred']
    argmax: [false, true]  # apply argmax to the prediction only to get a class index number
    to_onehot: 6  # convert both prediction and label to one-hot format so that both have shape (6,)

# separating out loss, inferer, and optimizer definitions

loss_function:
  _target_: torch.nn.CrossEntropyLoss

inferer: 
  _target_: SimpleInferer

optimizer: 
  _target_: torch.optim.Adam
  params: '$@net.parameters()'
  lr: '@learning_rate'

# Handlers to load the checkpoint if present, run validation at the chosen interval, save the checkpoint
# at the chosen interval, log stats, and write the log to a file in the output directory.
handlers:
- _target_: CheckpointLoader
  _disabled_: '$not os.path.exists(@ckpt_path)'
  load_path: '@ckpt_path'
  load_dict:
    model: '@net'
- _target_: ValidationHandler
  validator: '@evaluator'
  epoch_level: true
  interval: '@val_interval'
- _target_: CheckpointSaver
  save_dir: '@output_dir'
  save_dict:
    model: '@net'
  save_interval: '@save_interval'
  save_final: true  # save the final weights, either when the run finishes or is interrupted somehow
- _target_: StatsHandler
  name: train_loss
  tag_name: train_loss
  output_transform: '$monai.handlers.from_engine([''loss''], first=True)'  # print per-iteration loss
- _target_: LogfileHandler
  output_dir: '@output_dir'

trainer:
  _target_: SupervisedTrainer
  device: '@device'
  max_epochs: '@max_epochs'
  train_data_loader: '@train_dl'
  network: '@net'
  optimizer: '@optimizer'
  loss_function: '@loss_function'
  inferer: '@inferer'
  train_handlers: '@handlers'

# validation handlers which log stats and direct the log to a file
val_handlers:
- _target_: StatsHandler
  name: val_stats
  output_transform: '$lambda x: None'
- _target_: LogfileHandler
  output_dir: '@output_dir'
    
# Metrics to assess validation results, you can have more than one here but may 
# need to adapt the format of pred and label.
metrics:
  accuracy:
    _target_: 'ignite.metrics.Accuracy'
    output_transform: '$monai.handlers.from_engine([@pred, @label])'

# runs the evaluation process, invoked by trainer via the ValidationHandler object
evaluator:
  _target_: SupervisedEvaluator
  device: '@device'
  val_data_loader: '@eval_dl'
  network: '@net'
  inferer: '@inferer'
  postprocessing: '@post_transform'
  key_val_metric: '@metrics'
  val_handlers: '@val_handlers'

train:
- '$@trainer.run()'


Overwriting MedNISTClassifier_v2/configs/train.yaml


We can now train as normal, specifying the logging config file and a maximum number of epochs you probably will want to set higher for a good result:

In [25]:
%%bash

BUNDLE="./MedNISTClassifier_v2"

python -m monai.bundle run train \
    --bundle_root "$BUNDLE" \
    --logging_file "$BUNDLE/configs/logging.conf" \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "['$BUNDLE/configs/common.yaml','$BUNDLE/configs/train.yaml']" \
    --max_epochs 2

2023-08-30 12:38:23,636 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-08-30 12:38:23,636 - INFO - > run_id: 'train'
2023-08-30 12:38:23,636 - INFO - > meta_file: './MedNISTClassifier_v2/configs/metadata.json'
2023-08-30 12:38:23,636 - INFO - > config_file: ['./MedNISTClassifier_v2/configs/common.yaml',
 './MedNISTClassifier_v2/configs/train.yaml']
2023-08-30 12:38:23,636 - INFO - > logging_file: './MedNISTClassifier_v2/configs/logging.conf'
2023-08-30 12:38:23,636 - INFO - > bundle_root: './MedNISTClassifier_v2'
2023-08-30 12:38:23,636 - INFO - > max_epochs: 2
2023-08-30 12:38:23,636 - INFO - ---


2023-08-30 12:38:23,636 - INFO - Setting logging properties based on config: ./MedNISTClassifier_v2/configs/logging.conf.
2023-08-30 12:38:23,768 - INFO - Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
2023-08-30 12:38:23,768 - INFO - File exists: MedNIST.tar.gz, skipped downloading.
2023-08-30 12:38:23,768 - INFO - Non-empty folder exists in MedNIST, ski

Loading dataset: 100%|██████████| 47164/47164 [00:41<00:00, 1134.34it/s]


2023-08-30 12:39:05,994 - INFO - Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
2023-08-30 12:39:05,994 - INFO - File exists: MedNIST.tar.gz, skipped downloading.
2023-08-30 12:39:05,994 - INFO - Non-empty folder exists in MedNIST, skipped extracting.


Loading dataset: 100%|██████████| 5895/5895 [00:05<00:00, 1135.59it/s]


2023-08-30 12:39:11,320 - ignite.engine.engine.SupervisedTrainer - INFO - Engine run resuming from iteration 0, epoch 0 until 2 epochs
2023-08-30 12:39:12,457 - INFO - Epoch: 1/2, Iter: 1/93 -- train_loss: 1.8415 
2023-08-30 12:39:12,828 - INFO - Epoch: 1/2, Iter: 2/93 -- train_loss: 1.8107 
2023-08-30 12:39:13,194 - INFO - Epoch: 1/2, Iter: 3/93 -- train_loss: 1.7766 
2023-08-30 12:39:13,569 - INFO - Epoch: 1/2, Iter: 4/93 -- train_loss: 1.7330 
2023-08-30 12:39:13,951 - INFO - Epoch: 1/2, Iter: 5/93 -- train_loss: 1.7159 
2023-08-30 12:39:14,326 - INFO - Epoch: 1/2, Iter: 6/93 -- train_loss: 1.6599 
2023-08-30 12:39:14,698 - INFO - Epoch: 1/2, Iter: 7/93 -- train_loss: 1.6619 
2023-08-30 12:39:15,068 - INFO - Epoch: 1/2, Iter: 8/93 -- train_loss: 1.6289 
2023-08-30 12:39:15,442 - INFO - Epoch: 1/2, Iter: 9/93 -- train_loss: 1.5839 
2023-08-30 12:39:15,813 - INFO - Epoch: 1/2, Iter: 10/93 -- train_loss: 1.5505 
2023-08-30 12:39:16,184 - INFO - Epoch: 1/2, Iter: 11/93 -- train_loss: 1.

Results and logs get put into unique timestamped directories:

In [26]:
!tree output/output_230830_123911/

output/output_230830_123911/
├── log.txt
├── model_epoch=1.pt
├── model_epoch=2.pt
└── model_final_iteration=186.pt

0 directories, 4 files


## Inference

What is also needed is an inference script which will apply a loaded network to every image in a given directory and write a result to a file or to the log output. For segmentation networks this should save generated segmentations to know locations, but for this classification network we'll stick to just printing results to the log. 

First thing to do is create a test directory with only a few test images so we can demonstrate inference quickly:

In [38]:
root_dir = "."  # assuming MedNIST was downloaded to the current directory
num_images = 20
dataset = MedNISTDataset(root_dir=root_dir, section="test", download=False)

!mkdir -p test_images

for i in range(num_images):
    filename = dataset[i]["image_meta_dict"]["filename_or_obj"]
    print(filename, "Label:", dataset[i]["label"])
    !cp {root_dir}/{filename} test_images

Loading dataset: 100%|██████████| 5895/5895 [00:03<00:00, 1771.10it/s]


MedNIST/AbdomenCT/001990.jpeg Label: 0
MedNIST/BreastMRI/007676.jpeg Label: 1
MedNIST/ChestCT/006763.jpeg Label: 3
MedNIST/CXR/001214.jpeg Label: 2
MedNIST/Hand/004427.jpeg Label: 4
MedNIST/HeadCT/003806.jpeg Label: 5
MedNIST/HeadCT/004638.jpeg Label: 5
MedNIST/CXR/005013.jpeg Label: 2
MedNIST/ChestCT/008275.jpeg Label: 3
MedNIST/BreastMRI/000630.jpeg Label: 1
MedNIST/BreastMRI/007547.jpeg Label: 1
MedNIST/BreastMRI/008425.jpeg Label: 1
MedNIST/AbdomenCT/003981.jpeg Label: 0
MedNIST/Hand/001130.jpeg Label: 4
MedNIST/BreastMRI/005118.jpeg Label: 1
MedNIST/CXR/006505.jpeg Label: 2
MedNIST/ChestCT/008218.jpeg Label: 3
MedNIST/HeadCT/005305.jpeg Label: 5
MedNIST/AbdomenCT/007871.jpeg Label: 0
MedNIST/Hand/007065.jpeg Label: 4


Next remove the existing example inference script:

In [5]:
!rm "MedNISTClassifier_v2/configs/inference.json"

Next we'll create the inference script which will apply the network to all the files in the given directory (thus assuming all are images) and save the results to a csv file:

In [90]:
%%writefile MedNISTClassifier_v2/configs/inference.yaml

imports:
- $import glob

input_dir: 'input'
# dataset is a list of dictionaries to work with dictionary transforms
input_files: '$[{@image: f} for f in sorted(glob.glob(@input_dir+''/*.*''))]'

infer_dataset:
  _target_: Dataset
  data: '@input_files'
  transform: 
    _target_: Compose
    transforms: '@train_transforms'

infer_dl:
  _target_: DataLoader
  dataset: '@infer_dataset'
  batch_size: 1
  shuffle: false
  num_workers: 0

# transforms applied to network output, same as those in training except "label" isn't present
post_transform:
  _target_: Compose
  transforms:
  - _target_: Activationsd
    keys: '@pred'
    softmax: true 
  - _target_: AsDiscreted
    keys: ['@pred']
    argmax: true 

# handlers to load the checkpoint file (and fail if a file isn't found), and save classification results to a csv file
handlers:
- _target_: CheckpointLoader
  load_path: '@ckpt_path'
  load_dict:
    model: '@net'
- _target_: ClassificationSaver
  batch_transform: '$lambda batch: batch[0][@image].meta'
  output_transform: '$monai.handlers.from_engine([''pred''])'

inferer: 
  _target_: SimpleInferer

evaluator:
  _target_: SupervisedEvaluator
  device: '@device'
  val_data_loader: '@infer_dl'
  network: '@net'
  inferer: '@inferer'
  postprocessing: '@post_transform'
  val_handlers: '@handlers'

inference:
- '$@evaluator.run()'

Overwriting MedNISTClassifier_v2/configs/inference.yaml


Inference can now be run, specifying the checkpoint file to load as being one from our training run and the input directory as "test_images" which was created above:

In [6]:
%%bash

BUNDLE="./MedNISTClassifier_v2"

python -m monai.bundle run inference \
    --bundle_root "$BUNDLE" \
    --logging_file "$BUNDLE/configs/logging.conf" \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "['$BUNDLE/configs/common.yaml','$BUNDLE/configs/inference.yaml']" \
    --ckpt_path 'output/output_230830_123911/model_final_iteration=186.pt' \
    --input_dir test_images 

2023-09-07 16:20:16,087 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-09-07 16:20:16,087 - INFO - > run_id: 'inference'
2023-09-07 16:20:16,087 - INFO - > meta_file: './MedNISTClassifier_v2/configs/metadata.json'
2023-09-07 16:20:16,087 - INFO - > config_file: ['./MedNISTClassifier_v2/configs/common.yaml',
 './MedNISTClassifier_v2/configs/inference.yaml']
2023-09-07 16:20:16,087 - INFO - > logging_file: './MedNISTClassifier_v2/configs/logging.conf'
2023-09-07 16:20:16,087 - INFO - > bundle_root: './MedNISTClassifier_v2'
2023-09-07 16:20:16,087 - INFO - > ckpt_path: 'output/output_230830_123911/model_final_iteration=186.pt'
2023-09-07 16:20:16,087 - INFO - > input_dir: 'test_images'
2023-09-07 16:20:16,087 - INFO - ---


2023-09-07 16:20:16,088 - INFO - Setting logging properties based on config: ./MedNISTClassifier_v2/configs/logging.conf.
2023-09-07 16:20:16,487 - ignite.engine.engine.SupervisedEvaluator - INFO - Engine run resuming from iteration 0, epoch 0 until 1 

This will save the results of the inference to "predictions.csv" by default. You can change what the output filename is with an argument like `'--handlers#1#filename' pred.csv` which will directly change the `filename` parameter of the appropriate handler. Note the single quotes around the argument name since the hash sigil is interpreted by Bash as a comment otherwise.

Looking at the output, the results aren't terribly legible:

In [88]:
!cat predictions.csv

test_images/000630.jpeg,1.0
test_images/001130.jpeg,4.0
test_images/001214.jpeg,2.0
test_images/001990.jpeg,0.0
test_images/003806.jpeg,5.0
test_images/003981.jpeg,0.0
test_images/004427.jpeg,4.0
test_images/004638.jpeg,5.0
test_images/005013.jpeg,2.0
test_images/005118.jpeg,1.0
test_images/005305.jpeg,5.0
test_images/006505.jpeg,2.0
test_images/006763.jpeg,3.0
test_images/007065.jpeg,4.0
test_images/007547.jpeg,1.0
test_images/007676.jpeg,1.0
test_images/007871.jpeg,0.0
test_images/008218.jpeg,3.0
test_images/008275.jpeg,3.0
test_images/008425.jpeg,1.0


The second column is the predicted class which we can use as an index into our list of class names to get something more readable:

In [121]:
class_names = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]

for fn, idx in np.loadtxt("predictions.csv", delimiter=",", dtype=str):
    print(fn, class_names[int(float(idx))])

test_images/000630.jpeg BreastMRI
test_images/001130.jpeg Hand
test_images/001214.jpeg CXR
test_images/001990.jpeg AbdomenCT
test_images/003806.jpeg HeadCT
test_images/003981.jpeg AbdomenCT
test_images/004427.jpeg Hand
test_images/004638.jpeg HeadCT
test_images/005013.jpeg CXR
test_images/005118.jpeg BreastMRI
test_images/005305.jpeg HeadCT
test_images/006505.jpeg CXR
test_images/006763.jpeg ChestCT
test_images/007065.jpeg Hand
test_images/007547.jpeg BreastMRI
test_images/007676.jpeg BreastMRI
test_images/007871.jpeg AbdomenCT
test_images/008218.jpeg ChestCT
test_images/008275.jpeg ChestCT
test_images/008425.jpeg BreastMRI


## Putting the Bundle Together

We have a checkpoint for our network which produces good results that we can now make the "official" shared weights for the bundle. We need to copy the checkpoint into the `models` directory and optionally produce a Torchscript version of the network. 

For the Torchscript convertion MONAI provides the `ckpt_export` program in the bundles submodule:

In [7]:
%%bash

BUNDLE="./MedNISTClassifier_v2"

cp "output/output_230830_123911/model_final_iteration=186.pt" "$BUNDLE/models/model.pt"

python -m monai.bundle ckpt_export \
    --bundle_root "$BUNDLE" \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "['$BUNDLE/configs/common.yaml','$BUNDLE/configs/inference.yaml']" \
    --net_id network_def \
    --key_in_ckpt model \
    --ckpt_file "$BUNDLE/models/model.pt" \
    --filepath "$BUNDLE/models/model.ts" 

which tree && tree "$BUNDLE"

2023-09-07 16:20:25,463 - INFO - --- input summary of monai.bundle.scripts.ckpt_export ---
2023-09-07 16:20:25,463 - INFO - > net_id: 'network_def'
2023-09-07 16:20:25,463 - INFO - > filepath: './MedNISTClassifier_v2/models/model.ts'
2023-09-07 16:20:25,463 - INFO - > meta_file: './MedNISTClassifier_v2/configs/metadata.json'
2023-09-07 16:20:25,463 - INFO - > config_file: ['./MedNISTClassifier_v2/configs/common.yaml',
 './MedNISTClassifier_v2/configs/inference.yaml']
2023-09-07 16:20:25,463 - INFO - > ckpt_file: './MedNISTClassifier_v2/models/model.pt'
2023-09-07 16:20:25,463 - INFO - > key_in_ckpt: 'model'
2023-09-07 16:20:25,463 - INFO - > bundle_root: './MedNISTClassifier_v2'
2023-09-07 16:20:25,463 - INFO - ---


2023-09-07 16:20:28,048 - INFO - exported to file: ./MedNISTClassifier_v2/models/model.ts.
./MedNISTClassifier_v2
├── configs
│   ├── common.yaml
│   ├── inference.yaml
│   ├── logging.conf
│   ├── metadata.json
│   └── train.yaml
├── docs
│   └── README.md
├── LICENSE
└──

This will have produced the `model.ts` file in `models` as shown here which can be loaded in Python without the bundle config scripts like any other Torchscript object.

The arguments for the `ckpt_export` command specify the components to use in the config files and the checkpoint:
* `bundle_root`, `meta_file`, and `config_file` are as in previous usages.
* `net_id` specifies the object in the config files which represents the network definition, ie. the instantiated network object.
* `key_in_ckpt` names the key under which the weights for the model are found in the checkpoint, this assumes the checkpoint is a dictionary which is what `CheckpointSaver` produces, if this file isn't a dictionary omit this argument.
* `ckpt_file` the name of the checkpoint file itself
* `filepath` the output filename to store the Torchscript object to.

## Summary and Next

This tutorial has covered MONAI Bundle best practices:
  * Separate common definition config files which are combined with specific application files
  * Separating out definitions in config files for easier reading and changes
  * Using Engine based classes for traning and validation
  * Simple training run management with uniquely-created results directories
  * Inference script to generate a results csv file containing predictions
  
The next tutorial will discuss creating bundles to wrap pre-existing Pytorch code so that you can get code into the bundle ecosystem without rewriting the world.